In [1]:
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
# ｄｆとして保存済みの前処理済みデータを読み込む
df2001_2023_higecut = pd.read_pickle("df2001_2023_higecut0218.pkl")

In [3]:
df2001_2023_higecut.shape

(537533, 55)

In [4]:
df2001_2023_higecut.head(2)

,年,月,日,回次,場所,日次,レース番号,レース名,クラスコード,芝・ダ,...,母の父馬名,毛色,生年月日,単勝オッズ,馬印1,レース印1,PCI,外部指数,外部指数差,外部指数順位
0,23,8,6,1,札幌,6,7,１勝ｸﾗｽ,23,ダ,...,サンデーサイレンス,黒鹿,190409,37.2,,,47.8,NaN,NaN,NaN
1,23,8,6,1,札幌,6,7,１勝ｸﾗｽ,23,ダ,...,ダイワメジャー,黒鹿,200313,21.3,,,46.4,NaN,NaN,NaN


In [5]:
#どのようなコラムがあるか表示
print(df2001_2023_higecut.columns.to_list())

['年', '月', '日', '回次', '場所', '日次', 'レース番号', 'レース名', 'クラスコード', '芝・ダ', 'コースコード', '距離', '馬場状態', '馬名', '性別', '年齢', '騎手名', '斤量', '頭数', '馬番', '確定着順', '入線着順', '異常コード', '着差タイム', '人気順', '走破タイム', '走破時計', 'Unnamed: 27', '通過順1', '通過順2', '通過順3', '通過順4', '上がり3Fタイム', '馬体重', '調教師', '所属地', '賞金', '血統登録番号', '騎手コード', '調教師コード', 'レースID', '現馬主名', '生産者名', '父馬名', '母馬名', '母の父馬名', '毛色', '生年月日', '単勝オッズ', '馬印1', 'レース印1', 'PCI', '外部指数', '外部指数差', '外部指数順位']


In [6]:
#不要な列の削除
columns=['年','月','日','場所','馬名','距離','走破タイム','レース番号','クラスコード','馬場状態','性別',"騎手名",'斤量','馬番','馬体重','上がり3Fタイム','調教師','所属地','現馬主名','生産者名','父馬名','母馬名','母の父馬名','毛色','生年月日','レースID','賞金']
df2001_2023_higecut=df2001_2023_higecut[columns]
df2001_2023_higecut.shape

(537533, 27)

In [7]:
#どのようなコラムがあるか表示
print(df2001_2023_higecut.columns.to_list())

['年', '月', '日', '場所', '馬名', '距離', '走破タイム', 'レース番号', 'クラスコード', '馬場状態', '性別', '騎手名', '斤量', '馬番', '馬体重', '上がり3Fタイム', '調教師', '所属地', '現馬主名', '生産者名', '父馬名', '母馬名', '母の父馬名', '毛色', '生年月日', 'レースID', '賞金']


In [8]:
#速度を計算した列を追加
df2001_2023_higecut['速度(m/s)'] = df2001_2023_higecut['距離']/df2001_2023_higecut['走破タイム']
df2001_2023_higecut.head(1)

,年,月,日,場所,馬名,距離,走破タイム,レース番号,クラスコード,馬場状態,...,現馬主名,生産者名,父馬名,母馬名,母の父馬名,毛色,生年月日,レースID,賞金,速度(m/s)
0,23,8,6,札幌,サンダビューク,1000,59.3,7,23,不,...,加藤信之,村下清志,ザファクター,エミリア,サンデーサイレンス,黒鹿,190409,123160706,800,16.863406


In [9]:
df2001_2023_higecut.shape

(537533, 28)

In [10]:
#馬ごとの生涯平均速度を計算
df2001_2023_higecut['馬ごとの平均速度(m/s)'] = df2001_2023_higecut.groupby('馬名')['速度(m/s)'].transform('mean')
df2001_2023_higecut.head(1)

,年,月,日,場所,馬名,距離,走破タイム,レース番号,クラスコード,馬場状態,...,生産者名,父馬名,母馬名,母の父馬名,毛色,生年月日,レースID,賞金,速度(m/s),馬ごとの平均速度(m/s)
0,23,8,6,札幌,サンダビューク,1000,59.3,7,23,不,...,村下清志,ザファクター,エミリア,サンデーサイレンス,黒鹿,190409,123160706,800,16.863406,16.661039


In [11]:
#おおよその予測の列を追加（そのレースの距離÷馬ごとの平均速度）
df2001_2023_higecut['仮予測'] =df2001_2023_higecut['距離'] /df2001_2023_higecut['馬ごとの平均速度(m/s)']
df2001_2023_higecut.head(1)

,年,月,日,場所,馬名,距離,走破タイム,レース番号,クラスコード,馬場状態,...,父馬名,母馬名,母の父馬名,毛色,生年月日,レースID,賞金,速度(m/s),馬ごとの平均速度(m/s),仮予測
0,23,8,6,札幌,サンダビューク,1000,59.3,7,23,不,...,ザファクター,エミリア,サンデーサイレンス,黒鹿,190409,123160706,800,16.863406,16.661039,60.020266


In [12]:
#計算し用済みの列を消去
df2001_2023_higecut=df2001_2023_higecut.drop({'速度(m/s)','馬ごとの平均速度(m/s)'},axis=1)
df2001_2023_higecut.head(1)

,年,月,日,場所,馬名,距離,走破タイム,レース番号,クラスコード,馬場状態,...,現馬主名,生産者名,父馬名,母馬名,母の父馬名,毛色,生年月日,レースID,賞金,仮予測
0,23,8,6,札幌,サンダビューク,1000,59.3,7,23,不,...,加藤信之,村下清志,ザファクター,エミリア,サンデーサイレンス,黒鹿,190409,123160706,800,60.020266


In [13]:
#データフレームの名前を変更して作業
dirt_maesyori=df2001_2023_higecut

# 生まれて何日目かの列を作成

In [14]:
# 年カラムを4桁の形式に変換
# 閾値によって加える数値を判断
# lamda関数内の閾値を25とする
# (2026年以降のデータが含まれる場合、閾値の変更が必要)
dirt_maesyori['年'] = dirt_maesyori['年'].apply(lambda x: x + 2000 if x < 25 else x + 1900)
dirt_maesyori.head(1)

,年,月,日,場所,馬名,距離,走破タイム,レース番号,クラスコード,馬場状態,...,現馬主名,生産者名,父馬名,母馬名,母の父馬名,毛色,生年月日,レースID,賞金,仮予測
0,2023,8,6,札幌,サンダビューク,1000,59.3,7,23,不,...,加藤信之,村下清志,ザファクター,エミリア,サンデーサイレンス,黒鹿,190409,123160706,800,60.020266


In [15]:
# 年,月,日カラムをそれぞれ文字列型に変更
dirt_maesyori['年']=dirt_maesyori['年'].astype(str)
dirt_maesyori['月']=dirt_maesyori['月'].astype(str)
dirt_maesyori['日']=dirt_maesyori['日'].astype(str)


# データ型の確認
# objectは文字列型や混在型
print(f"年のデータ型：{dirt_maesyori['年'].dtype}")
print(f"月のデータ型：{dirt_maesyori['月'].dtype}")
print(f"日のデータ型：{dirt_maesyori['日'].dtype}")

年のデータ型：object
月のデータ型：object
日のデータ型：object


In [16]:
# '月' と '日' のカラムの各要素を2桁になるように0で埋める
dirt_maesyori['月'] = dirt_maesyori['月'].astype(str).str.zfill(2)
dirt_maesyori['日'] = dirt_maesyori['日'].astype(str).str.zfill(2)

In [17]:
# 処理の確認のため
# カラムそれぞれを一意の値にして列に格納
df_year = dirt_maesyori['年'].unique()
df_month = dirt_maesyori['月'].unique()
df_day = dirt_maesyori['日'].unique()

In [18]:
#　年の内容を確認
df_year.sort()
df_year

array(['2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2020', '2021', '2022', '2023'],
      dtype=object)

In [19]:
#　月の内容を確認
df_month.sort()
df_month

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12'], dtype=object)

In [20]:
#　日の内容を確認
df_day.sort()
df_day

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31'], dtype=object)

In [21]:
# pd.to_datetime を使用して新しい 'date' カラムを作成
dirt_maesyori['date'] = pd.to_datetime({
    'year': dirt_maesyori['年'],
    'month': dirt_maesyori['月'],
    'day': dirt_maesyori['日']
})

# '年', '月', '日' の列をドロップ
dirt_maesyori = dirt_maesyori.drop(['年', '月', '日'], axis=1)

#　列の追加と削除の確認
dirt_maesyori.head()

,場所,馬名,距離,走破タイム,レース番号,クラスコード,馬場状態,性別,騎手名,斤量,...,生産者名,父馬名,母馬名,母の父馬名,毛色,生年月日,レースID,賞金,仮予測,date
0,札幌,サンダビューク,1000,59.3,7,23,不,牡,石川倭,58.0,...,村下清志,ザファクター,エミリア,サンデーサイレンス,黒鹿,190409,123160706,800,60.020266,2023-08-06
1,札幌,フクノワカバ,1000,59.3,7,23,不,牝,鮫島克駿,54.0,...,信岡牧場,ドレフォン,マレーナ,ダイワメジャー,黒鹿,200313,123160705,320,60.690153,2023-08-06
2,札幌,クロンヌドラレーヌ,1000,59.3,7,23,不,牝,佐々木大,54.0,...,社台ファーム,ロードカナロア,クラウンプリンセス,スペシャルウィーク,鹿毛,190510,123160712,200,62.369290,2023-08-06
3,札幌,アカザ,1000,59.4,7,23,不,牝,鷲頭虎太,53.0,...,上水牧場,ミッキーアイル,マイネハッピー,ブライアンズタイム,鹿毛,190307,123160710,120,59.487523,2023-08-06
4,札幌,ナムラジョシュア,1000,59.5,7,23,不,牡,長岡禎仁,56.0,...,中島牧場,シニスターミニスター,ナムラネネ,ケイムホーム,黒鹿,200411,123160704,80,61.181628,2023-08-06


In [22]:
# 生年月日カラムを文字列型に変換
dirt_maesyori['生年月日'] = dirt_maesyori['生年月日'].astype(str)

In [23]:
for i in range(2,9):
    # '生年月日'カラムの各要素を文字列に変換し、その長さが1である行の数を数える
    count = dirt_maesyori[dirt_maesyori['生年月日'].astype(str).apply(len) == i].count()
    # 結果を "i文字: count" の形式で表示
    print(f"{i}文字: {count['生年月日']}件")

2文字: 0件
3文字: 24565件
4文字: 11件
5文字: 211540件
6文字: 301417件
7文字: 0件
8文字: 0件


In [24]:
# 生年月日の文字列の長さごとの内容を確認

for i in range(2,9):
    invalid_birthday_rows =dirt_maesyori[dirt_maesyori['生年月日'].str.len() == i]
    print(f"{i}文字の内容：{invalid_birthday_rows['生年月日'].unique()}")
    print(f"{i}文字の要素の数＜一意＞：{len(invalid_birthday_rows['生年月日'].unique())}")

2文字の内容：[]
2文字の要素の数＜一意＞：0
3文字の内容：['315' '311' '525' '409' '428' '404' '513' '202' '412' '313' '224' '418'
 '419' '504' '414' '310' '328' '430' '205' '417' '408' '217' '421' '207'
 '221' '324' '420' '505' '308' '326' '511' '226' '209' '323' '426' '503'
 '307' '331' '131' '316' '407' '330' '222' '314' '422' '327' '129' '429'
 '201' '423' '401' '506' '220' '410' '519' '424' '303' '516' '512' '213'
 '517' '305' '318' '507' '203' '413' '416' '531' '510' '607' '312' '427'
 '522' '403' '216' '329' '306' '415' '515' '425' '405' '225' '930' '529'
 '411' '520' '215' '319' '211' '524' '322' '121' '523' '325' '304' '301'
 '214' '528' '223' '228' '212' '122' '509' '527' '406' '321' '501' '309'
 '402' '514' '113' '204' '229' '530' '518' '616' '302' '116' '320' '210'
 '502' '118' '526' '219' '208' '508' '119' '606' '123' '613' '605' '612'
 '602' '601' '610' '521' '809' '124' '317' '615' '218' '609' '619' '127'
 '604' '206' '603' '125' '705' '622' '227' '126' '614' '120' '110' '608'
 '620' '130' '617' 

In [25]:
# 生年月日の文字数に応じた処理を関数で定義

# 3桁：文字列20000を前に連結
# 4桁：文字列2000を前に連結
# 5桁：文字列200を前に連結
# 6桁：先頭が9である場合は前に文字列19を連結、それ以外は前に文字列20を連結

def process_date(x):
    if len(x) == 3:
        return '20000' + x
    elif len(x) == 4:
        return '2000' + x
    elif len(x) == 5:
        return '200' + x
    elif len(x) == 6:
        return '19' + x if x.startswith('9') else '20' + x
    else:
        return x  # 他のケースでは元の値を返す

# '生年月日'カラムの値を加工
#  applyメソッドは引数へ与えた関数を各値に処理して返す
dirt_maesyori['生年月日'] = dirt_maesyori['生年月日'].astype(str).apply(process_date)

In [26]:
# 処理が施されているかの確認
# 生年月日の文字列の長さごとの内容を確認

for i in range(2,9):
    invalid_birthday_rows = dirt_maesyori[dirt_maesyori['生年月日'].str.len() == i]
    print(f"{i}文字の内容：{invalid_birthday_rows['生年月日'].unique()}")
    print(f"{i}文字の要素の数＜一意＞：{len(invalid_birthday_rows['生年月日'].unique())}")

2文字の内容：[]
2文字の要素の数＜一意＞：0
3文字の内容：[]
3文字の要素の数＜一意＞：0
4文字の内容：[]
4文字の要素の数＜一意＞：0
5文字の内容：[]
5文字の要素の数＜一意＞：0
6文字の内容：[]
6文字の要素の数＜一意＞：0
7文字の内容：[]
7文字の要素の数＜一意＞：0
8文字の内容：['20190409' '20200313' '20190510' ... '19970924' '19970619' '19940504']
8文字の要素の数＜一意＞：4464


In [27]:
# pd.to_datetime で生年月日をdatetime型に変換
dirt_maesyori['生年月日'] = pd.to_datetime(dirt_maesyori['生年月日'])

In [28]:
# 生年月日のデータ型が変換されているか確認
print(f"生年月日のデータ型：{dirt_maesyori['生年月日'].dtype}")

生年月日のデータ型：datetime64[ns]


In [29]:
# 'date'から'生年月日'を引いて日数を計算し、'Age_in_Days'カラムに追加
dirt_maesyori['生後日数'] = dirt_maesyori.apply(lambda row: (row['date'] - row['生年月日']).days, axis=1)

# 追加カラムの確認
dirt_maesyori.head(3)

,場所,馬名,距離,走破タイム,レース番号,クラスコード,馬場状態,性別,騎手名,斤量,...,父馬名,母馬名,母の父馬名,毛色,生年月日,レースID,賞金,仮予測,date,生後日数
0,札幌,サンダビューク,1000,59.3,7,23,不,牡,石川倭,58.0,...,ザファクター,エミリア,サンデーサイレンス,黒鹿,2019-04-09,123160706,800,60.020266,2023-08-06,1580
1,札幌,フクノワカバ,1000,59.3,7,23,不,牝,鮫島克駿,54.0,...,ドレフォン,マレーナ,ダイワメジャー,黒鹿,2020-03-13,123160705,320,60.690153,2023-08-06,1241
2,札幌,クロンヌドラレーヌ,1000,59.3,7,23,不,牝,佐々木大,54.0,...,ロードカナロア,クラウンプリンセス,スペシャルウィーク,鹿毛,2019-05-10,123160712,200,62.369290,2023-08-06,1549


In [30]:
#用済み列を削除
dirt_maesyori=dirt_maesyori.drop({'生年月日','date'},axis=1)
dirt_maesyori.head(3)

,場所,馬名,距離,走破タイム,レース番号,クラスコード,馬場状態,性別,騎手名,斤量,...,現馬主名,生産者名,父馬名,母馬名,母の父馬名,毛色,レースID,賞金,仮予測,生後日数
0,札幌,サンダビューク,1000,59.3,7,23,不,牡,石川倭,58.0,...,加藤信之,村下清志,ザファクター,エミリア,サンデーサイレンス,黒鹿,123160706,800,60.020266,1580
1,札幌,フクノワカバ,1000,59.3,7,23,不,牝,鮫島克駿,54.0,...,佐藤範夫,信岡牧場,ドレフォン,マレーナ,ダイワメジャー,黒鹿,123160705,320,60.690153,1241
2,札幌,クロンヌドラレーヌ,1000,59.3,7,23,不,牝,佐々木大,54.0,...,ライオンレースホース,社台ファーム,ロードカナロア,クラウンプリンセス,スペシャルウィーク,鹿毛,123160712,200,62.369290,1549


# レースごとの賞金（ここでの意味は懸賞金）列を作成

In [31]:
# '一意なレースID'列を作成する

# 特定の列のデータ型をintからstrに変換する
dirt_maesyori['レースID'] = dirt_maesyori['レースID'].astype(str)

# 特定の列の文字列の下2桁を削除する関数
def remove_last_two_chars(text):
    return text[:-2]

# applyメソッドを使用して特定の列の各セルに関数を適用する
dirt_maesyori['一意なレースID'] = dirt_maesyori['レースID'].apply(remove_last_two_chars)

dirt_maesyori[10:14]

,場所,馬名,距離,走破タイム,レース番号,クラスコード,馬場状態,性別,騎手名,斤量,...,生産者名,父馬名,母馬名,母の父馬名,毛色,レースID,賞金,仮予測,生後日数,一意なレースID
10,札幌,コスモジョセフ,1000,60.0,7,23,不,牡,丹内祐次,56.0,...,シンユウフアーム,アポロキングダム,ジョリーン,トワイニング,鹿毛,123160709,0,60.777168,1222,1231607
11,札幌,メイム,1000,60.7,7,23,不,牝,山田敬士,56.0,...,川上牧場,ミッキーアイル,ケージーメグミ,キングカメハメハ,鹿毛,123160708,0,61.364521,1905,1231607
12,札幌,ダラムキャッスル,1000,57.8,2,7,不,牝,横山琉人,53.0,...,下河辺牧場,ヘニーヒューズ,シュンドルボン,ハーツクライ,黒鹿,123150204,550,60.742200,1239,1231502
13,札幌,エイシンエマーユ,1000,58.4,2,7,不,牝,松田大作,54.0,...,栄進牧場,エイシンヒカリ,エイシンピカソ,Drosselmeyer,鹿毛,123150205,220,58.400000,1255,1231502


In [32]:
#どのようなコラムがあるか表示
print(dirt_maesyori.columns.to_list())

['場所', '馬名', '距離', '走破タイム', 'レース番号', 'クラスコード', '馬場状態', '性別', '騎手名', '斤量', '馬番', '馬体重', '上がり3Fタイム', '調教師', '所属地', '現馬主名', '生産者名', '父馬名', '母馬名', '母の父馬名', '毛色', 'レースID', '賞金', '仮予測', '生後日数', '一意なレースID']


In [33]:
#レースごとの平均賞金を計算
dirt_maesyori['レースごとの平均賞金'] = dirt_maesyori.groupby('一意なレースID')['賞金'].transform('mean')
dirt_maesyori[10:14]

,場所,馬名,距離,走破タイム,レース番号,クラスコード,馬場状態,性別,騎手名,斤量,...,父馬名,母馬名,母の父馬名,毛色,レースID,賞金,仮予測,生後日数,一意なレースID,レースごとの平均賞金
10,札幌,コスモジョセフ,1000,60.0,7,23,不,牡,丹内祐次,56.0,...,アポロキングダム,ジョリーン,トワイニング,鹿毛,123160709,0,60.777168,1222,1231607,126.666667
11,札幌,メイム,1000,60.7,7,23,不,牝,山田敬士,56.0,...,ミッキーアイル,ケージーメグミ,キングカメハメハ,鹿毛,123160708,0,61.364521,1905,1231607,126.666667
12,札幌,ダラムキャッスル,1000,57.8,2,7,不,牝,横山琉人,53.0,...,ヘニーヒューズ,シュンドルボン,ハーツクライ,黒鹿,123150204,550,60.742200,1239,1231502,87.333333
13,札幌,エイシンエマーユ,1000,58.4,2,7,不,牝,松田大作,54.0,...,エイシンヒカリ,エイシンピカソ,Drosselmeyer,鹿毛,123150205,220,58.400000,1255,1231502,87.333333


In [34]:
#用済み列を削除
dirt_maesyori=dirt_maesyori.drop({'賞金'},axis=1)
dirt_maesyori.head(3)

,場所,馬名,距離,走破タイム,レース番号,クラスコード,馬場状態,性別,騎手名,斤量,...,生産者名,父馬名,母馬名,母の父馬名,毛色,レースID,仮予測,生後日数,一意なレースID,レースごとの平均賞金
0,札幌,サンダビューク,1000,59.3,7,23,不,牡,石川倭,58.0,...,村下清志,ザファクター,エミリア,サンデーサイレンス,黒鹿,123160706,60.020266,1580,1231607,126.666667
1,札幌,フクノワカバ,1000,59.3,7,23,不,牝,鮫島克駿,54.0,...,信岡牧場,ドレフォン,マレーナ,ダイワメジャー,黒鹿,123160705,60.690153,1241,1231607,126.666667
2,札幌,クロンヌドラレーヌ,1000,59.3,7,23,不,牝,佐々木大,54.0,...,社台ファーム,ロードカナロア,クラウンプリンセス,スペシャルウィーク,鹿毛,123160712,62.369290,1549,1231607,126.666667


In [35]:
# データフレームをpickleファイルとして保存
dirt_maesyori.to_pickle('dirt_maesyori0218.pkl')

In [36]:
# データフレームをExcelファイルとして保存
#dirt_maesyori.to_excel(r'C:\Users\5D170\Desktop\Python\競馬チーム\dirt_maesyori0218.xlsx')